In [133]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import pandas as pd 

In [134]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


2022-07-09 00:02:35,914 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-09 00:02:36,057 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


2022-07-09 00:02:36,058 INFO Get LATEST chromedriver version for 103.0.5060 google-chrome


[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_mac64.zip


2022-07-09 00:02:36,987 INFO About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/origi/.wdm/drivers/chromedriver/mac64/103.0.5060.53]


2022-07-09 00:02:40,292 INFO Driver has been saved in cache [/Users/origi/.wdm/drivers/chromedriver/mac64/103.0.5060.53]


### Visit the NASA Mars News Site

In [17]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [18]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [19]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's Perseverance Mars Rover Gets Balanced"

In [20]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"The mission team performed a crucial weight-balancing test on the rover in preparation for this summer's history-making launch to the Red Planet."

### JPL Space Images Featured Image

In [7]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [8]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [9]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [10]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

### Mars Facts

In [1]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

NameError: name 'img_url_rel' is not defined

In [13]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

2022-07-07 22:02:12,400 INFO NumExpr defaulting to 8 threads.


,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [129]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
for i in range(4):
    
    # 1. Use browser to visit the URL 
    url = 'https://marshemispheres.com/'
    browser.visit(url)
    
    # Parse the resulting html with soup
    html = browser.html
    hemi_soup = soup(html, 'html.parser')
    hemi_title_elem = hemi_soup.find_all('div', class_='description')[i]
    
    # Obtaining the hemisphere title
    hemi_title = hemi_title_elem.find('a', class_='itemLink product-item').get_text(separator='\n').strip()
    hemi_title
    
    # Parse to find the thumbnail image and finding the href
    # I couldn't find a botton for browser to click into
    # Instead I found the href and built another url for browser to visit
    hemi_href = hemi_title_elem.find('a', class_='itemLink product-item')['href']
    hemi_url = f'https://marshemispheres.com/{hemi_href}'
    hemi_url

    # Onto the next step by getting browser to visit the url from above
    sample_image_elem = browser.visit(hemi_url)
    
    # Parsing the new url link/ I don't know if this is incorrect but it worked
    html = browser.html
    sample_soup = soup(html, 'html.parser')
    sample_elem = sample_soup.find('div', class_='downloads') # Setting up to get image url 
    sample_target = sample_elem.find('a', href=True)['href'] # Getting href to build url
    sample_url = f'https://marshemispheres.com/{sample_target}'
    sample_url
    
    hemisphere_image_urls.append({'img_url': sample_url, 'title': hemi_title})


In [131]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [138]:
# 5. Quit the browser
browser.quit()